In [ ]:
!mkdir  -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d grassknoted/asl-alphabet -p /content/data --unzip

Dataset URL: https://www.kaggle.com/datasets/grassknoted/asl-alphabet
License(s): GPL-2.0
100% 1.02G/1.03G [00:07<00:00, 302MB/s]
100% 1.03G/1.03G [00:07<00:00, 145MB/s]


In [ ]:
from torchvision.datasets import ImageFolder
import os
data_path = "/content/data/asl_alphabet_train"
full_dataset = ImageFolder(root=data_path)

In [ ]:
from sklearn.model_selection import train_test_split
num_samples = len(full_dataset)
indices = list(range(num_samples))
targets = [s[1] for s in full_dataset.samples]

trainval_idx, test_idx = train_test_split(
    indices, test_size=0.10, stratify=targets, random_state=42
)

train_idx, val_idx = train_test_split(
    trainval_idx,
    test_size=0.2222,
    stratify=[targets[i] for i in trainval_idx],
    random_state=42
)

print(f"Train: {len(train_idx)}, Val: {len(val_idx)}, Test: {len(test_idx)}")

Train: 60901, Val: 17399, Test: 8700


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0

def find_class_folder(root):
    if not os.path.isdir(root):
        raise FileNotFoundError(root)
    entries = [e for e in os.listdir(root) if os.path.isdir(os.path.join(root,e))]
    # if root already contains class folders (like 'A','B',...) return root
    if any(len(name)==1 and name.isalpha() for name in entries) or len(entries) >= 10:
        return root
    # otherwise check one level deeper and return first candidate with many subfolders
    for e in entries:
        p = os.path.join(root, e)
        sub = [s for s in os.listdir(p) if os.path.isdir(os.path.join(p,s))]
        if any(len(name)==1 and name.isalpha() for name in sub) or len(sub) >= 10:
            return p
    # fallback to root
    return root

base_train = "/content/data/asl_alphabet_train"
base_test  = "/content/data/asl_alphabet_test"

train_path = find_class_folder(base_train)
test_path  = find_class_folder(base_test)

print("Using train_path:", train_path)
print("Using test_path :", test_path)

IMG_SIZE = (128,128)
BATCH_SIZE = 64
EPOCHS = 2

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path, validation_split=0.4, subset="training", seed=42,
    image_size=IMG_SIZE, batch_size=BATCH_SIZE
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path, validation_split=0.4, subset="validation", seed=42,
    image_size=IMG_SIZE, batch_size=BATCH_SIZE
)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_path, image_size=IMG_SIZE, batch_size=BATCH_SIZE
)

print("Classes detected:", train_ds.class_names)

base_model = EfficientNetB0(include_top=False, input_shape=IMG_SIZE+(3,), weights="imagenet")
base_model.trainable = False

model = models.Sequential([base_model, layers.GlobalAveragePooling2D(), layers.Dense(len(train_ds.class_names), activation="softmax")])
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)
loss, acc = model.evaluate(test_ds)
print("Test acc:", acc)
model.save("asl_model.h5")

Using train_path: /content/data/asl_alphabet_train/asl_alphabet_train
Using test_path : /content/data/asl_alphabet_test
Found 87000 files belonging to 29 classes.
Using 52200 files for training.
Found 87000 files belonging to 29 classes.
Using 34800 files for validation.
Found 28 files belonging to 1 classes.
Classes detected: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
Epoch 1/2
816/816 ━━━━━━━━━━━━━━━━━━━━ 2041s 2s/step - accuracy: 0.7349 - loss: 1.2262 - val_accuracy: 0.9770 - val_loss: 0.1927
Epoch 2/2
816/816 ━━━━━━━━━━━━━━━━━━━━ 2009s 2s/step - accuracy: 0.9673 - loss: 0.2054 - val_accuracy: 0.9889 - val_loss: 0.0968
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 945ms/step - accuracy: 0.0357 - loss: 15.9469


Test acc: 0.0357142873108387


In [ ]:
# Run in Colab (or local) where TensorFlow version matches your training env
import tensorflow as tf

# adjust filename if different
MODEL_IN = "asl_model.keras"   # or "asl_model.h5" if that's what you have in Colab
WEIGHTS_OUT = "asl_weights.weights.h5"

m = tf.keras.models.load_model(MODEL_IN, compile=False)
m.save_weights(WEIGHTS_OUT)
print("Saved weights to", WEIGHTS_OUT)

Saved weights to asl_weights.weights.h5


In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model("asl_model.h5", compile=False)
model.save("asl_model_portable.keras")   # saves everything


In [ ]:
# Re-save a portable .keras model (do this in your Colab where training ran)
import tensorflow as tf
import os

# adjust if your file is different
IN = "asl_model.h5"   # or "asl_model.keras" if you already have it
OUT = "asl_model_portable.keras"

print("Loading", IN)
m = tf.keras.models.load_model(IN, compile=False)
print("Saving portable model to", OUT)
m.save(OUT)   # creates a single .keras file that includes architecture + weights

# show file size to verify upload-able file
print("Size:", os.path.getsize(OUT), "bytes")

Loading asl_model.h5
Saving portable model to asl_model_portable.keras
Size: 17175304 bytes
